este testea, sin re-entrenar, just con los pesos guardados del modelo after training

In [2]:
import os, sys
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.nn.functional as F

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import util, models

In [4]:
audio_path = os.path.join('audio.ogg')
if not os.path.exists(audio_path):
    print(f"Audio file not found at {audio_path}. Please provide a valid path.")
    sys.exit(1)

print(audio_path)

audio.ogg


In [ ]:
audio_matrices = util.audio_process(audio_path, reduce_noise=True, sr=32000, segment_sec=5.0, frame_len=2048, hop_len=512, mels=224, nfft=2048)

if audio_matrices is None:
    print("Failed to process audio file. Please check the file format and content.")
    sys.exit(1)

In [ ]:
print(len(audio_matrices))

print(audio_matrices[0].shape)
print(audio_matrices[1].shape)
print(audio_matrices[2].shape)
print(audio_matrices[-1].shape)